In [47]:
from config import data_raw_dir, data_processed_dir
from os.path import join
import pandas as pd
from geopy.geocoders import Nominatim

In [48]:
datasets_dir = join(data_raw_dir, "Hackacity 23", "Datasets")
alerts_path = join(datasets_dir, "Road Alerts", "alerts_data", "alerts.json")

In [49]:
alerts = pd.read_json(alerts_path)

In [50]:
alerts

,id,type,alertSource,category,dataProvider,dateIssued,description,location,severity,source,subCategory,validFrom,validTo
0,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-02-12T00:25:00.00Z,Rua do Aleixo Nº72 - Avaria de telecomunicaçõe...,"[-8.649091427, 41.149767771]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-02-15T13:00:00.00Z,2022-02-15T16:00:00.00Z
1,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-02-12T00:27:00.00Z,Rua de João de Barros Nº361 - Avaria de teleco...,"[-8.663709507, 41.15583859]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-02-16T10:00:00.00Z,2022-02-16T12:30:00.00Z
2,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-02-12T00:35:00.00Z,Rua de Alberto de Serpa Nº86 - Avaria de telec...,"[-8.656609993, 41.164796971]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-02-17T10:00:00.00Z,2022-02-17T12:30:00.00Z
3,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-02-14T22:43:00.00Z,Rua de Alcântara 491 - Outro motivo,"[-8.584118224000001, 41.172921389]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-02-15T08:00:00.00Z,2022-02-15T23:45:00.00Z
4,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-02-14T22:45:00.00Z,Rua do Bonfim 318 - Outro motivo,"[-8.595385639, 41.150134039]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-02-15T08:00:00.00Z,2022-02-15T23:45:00.00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-12-30T11:28:00.00Z,Rua das Motas 71 - Avaria elétrica,"[-8.670882321, 41.149558131]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-12-30T09:30:00.00Z,2022-12-30T23:45:00.00Z
11898,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-12-30T11:32:00.00Z,Rua de Pedro Hispano 1246 - Avaria elétrica,"[-8.628857135, 41.165889641]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,roadWorks,2022-12-30T09:30:00.00Z,2022-12-30T23:45:00.00Z
11899,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-12-31T18:39:00.00Z,Rua de Fernandes Tomás proximo ao campo 24 Ago...,"[-8.600129838, 41.148594038]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,carAccident,2022-12-31T18:45:00.00Z,2022-12-31T00:00:00.00Z
11900,urn:ngsi-ld:Alert:porto:RoadAlertsUnscheduled:...,Alert,Câmara Municipal do Porto,traffic,https://www.cm-porto.pt/,2022-12-31T09:21:00.00Z,Rua de Coutinho de Azevedo 38 - Rotura de conduta,"[-8.594954307, 41.157024507]",informational,https://servsig.cm-porto.pt/arcgis/rest/servic...,pothole,2022-12-31T09:15:00.00Z,2022-12-31T23:45:00.00Z


In [51]:
datetime_columns = ['dateIssued', 'validFrom', 'validTo']
for col in datetime_columns:
    alerts[col] = pd.to_datetime(alerts[col])

In [52]:
# Filter based on a start and end date
start_date = '2022-07-02'
end_date = '2022-12-31'

# Convert start and end date strings to timestamp objects
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)

# Filter the DataFrame based on the date range
filtered_alerts = alerts[(alerts['validFrom'].dt.tz_localize(None) >= start_date) & (alerts['validTo'].dt.tz_localize(None) <= end_date)]


In [53]:
filtered_alerts.shape

(5241, 13)

In [54]:
def get_closest_street_osm_id(latitude, longitude):
    geolocator = Nominatim(user_agent="closest_street_finder")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)

    # Extracting the OSM ID of the closest street
    osm_id = location.raw['osm_id']

    return osm_id

# Example coordinates (New York City)
lat = 40.7128
lon = -74.0060

closest_street_osm_id = get_closest_street_osm_id(lat, lon)
print(f"The OSM ID of the closest street is: {closest_street_osm_id}")


The OSM ID of the closest street is: 575213527


In [55]:
def get_osm_id(coords):
    return get_closest_street_osm_id(coords[1], coords[0])

In [56]:
# Create a new column 'osm_id' using apply
alerts['osm_id'] = alerts['location'].apply(get_osm_id)

In [ ]:
alerts